# Logistic Regression - Proje: Müşteri Churn Tahmini

Bu proje, bir telekomünikasyon şirketinin müşteri kaybını (churn) tahmin etmek için Logistic Regression kullanır.


## Proje Adımları

1. Veri Keşfi ve Analiz
2. Veri Ön İşleme
3. Feature Engineering
4. Model Eğitimi
5. Model Değerlendirme
6. Sonuçlar ve Öneriler


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

print("Kütüphaneler yüklendi!")


## 1. Veri Yükleme ve Keşif

Veri setini yükleyin ve temel istatistikleri inceleyin.


In [ ]:
# Veri setini yükleyin (örnek: kaggle'dan telecom churn dataset)
# df = pd.read_csv('telecom_churn.csv')

# VEYA örnek veri oluşturun
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=10000, n_features=20, n_informative=15, 
                           n_redundant=5, n_clusters_per_class=1, 
                           random_state=42, class_sep=0.8)

# Örnek feature isimleri
feature_names = [f'Feature_{i+1}' for i in range(20)]
df = pd.DataFrame(X, columns=feature_names)
df['Churn'] = y

print(f"Veri seti boyutu: {df.shape}")
print(f"\\nSınıf dağılımı:\\n{df['Churn'].value_counts()}")
print(f"\\nEksik değerler:\\n{df.isnull().sum().sum()}")
df.head()


## 2. Veri Ön İşleme ve Feature Engineering


In [ ]:
# X ve y'yi ayır
X = df.drop('Churn', axis=1)
y = df['Churn']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Eğitim seti boyutu: {X_train_scaled.shape}")
print(f"Test seti boyutu: {X_test_scaled.shape}")


## 3. Model Eğitimi


In [ ]:
# Model oluştur ve eğit
model = LogisticRegression(random_state=42, max_iter=1000, class_weight='balanced')
model.fit(X_train_scaled, y_train)

# Cross-validation
cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='accuracy')
print(f"Cross-validation Accuracy: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

# Tahmin
y_pred = model.predict(X_test_scaled)
y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]


## 4. Model Değerlendirme


In [ ]:
# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print(f"\\nConfusion Matrix:\\n{cm}")

# Classification Report
print(f"\\nClassification Report:\\n{classification_report(y_test, y_pred)}")

# ROC-AUC
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"\\nROC-AUC Score: {roc_auc:.4f}")

# ROC Curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()


## 5. Feature Importance

Model katsayılarını analiz ederek en önemli feature'ları belirleyin.


In [ ]:
# Feature importance (katsayıların mutlak değerleri)
feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': model.coef_[0],
    'Abs_Coefficient': np.abs(model.coef_[0])
}).sort_values('Abs_Coefficient', ascending=False)

print("En önemli 10 feature:")
print(feature_importance.head(10))

# Görselleştir
plt.figure(figsize=(10, 6))
sns.barplot(data=feature_importance.head(10), x='Abs_Coefficient', y='Feature')
plt.title('Top 10 Feature Importance')
plt.xlabel('Absolute Coefficient Value')
plt.show()


## 6. Sonuçlar ve Öneriler

Model performansını özetleyin ve iş değeri sağlayacak öneriler sunun.


In [ ]:
print("=" * 50)
print("PROJE ÖZETİ")
print("=" * 50)
print(f"\\nModel Accuracy: {accuracy:.4f}")
print(f"ROC-AUC Score: {roc_auc:.4f}")
print(f"\\nEn önemli 5 feature:")
for idx, row in feature_importance.head(5).iterrows():
    print(f"  - {row['Feature']}: {row['Coefficient']:.4f}")

print("\\n" + "=" * 50)
print("ÖNERİLER:")
print("=" * 50)
print("1. Model performansını artırmak için feature engineering yapılabilir")
print("2. Dengesiz veri seti için SMOTE veya diğer teknikler kullanılabilir")
print("3. Hyperparameter tuning ile model optimize edilebilir")
print("4. Ensemble methods ile daha iyi sonuçlar elde edilebilir")
